In [1]:
#transformation du texte sous forme de liste en string
def jonction(liste):
  result = ""
  for token in liste:
    result = result + token + " "
  return result

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from joblib import load,dump
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def fonction_vectorisation_countvectorizer(joblib,colonne,max_features):
  #import du df
  df = load(joblib)
  #transformation de la future colonne feature en chaine de caractère
  df[colonne]=df[colonne].apply(jonction)
  #séparation des features et de la cible
  data = df[colonne]
  target = df["prdtypecode"]
  #séparation de train et test
  X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify = target)
  #vectorisation selon un nombre max de mots les plus fréquents
  vectorizer = CountVectorizer(max_features=max_features)
  X_train = vectorizer.fit_transform(X_train).todense()
  X_test = vectorizer.transform(X_test).todense()
  scaler = StandardScaler().fit(np.asarray(X_train))
  X_train = scaler.transform(np.asarray(X_train))
  X_test = scaler.transform(np.asarray(X_test))
  return X_train, X_test, y_train, y_test,vectorizer,df

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest
import time
import os
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(n_jobs = -1,random_state = 23,class_weight="balanced")

def recherche_maxfeatures_kbest(joblib_path,colonne_a_vectorise,liste_max_features,liste_kbest):
  if os.path.exists(joblib_path):
    df_import = load(joblib_path)
    print("récupération du df existant")
  else:
    df_import = pd.DataFrame(columns=["Max_features","K_best", "Accuracy", "F1_weighted", "F1_macro", "Duree en sec"])
    print("création d'un dataframe") 
  score = []
  for i in liste_max_features:
    X_trainf, X_testf, y_trainf, y_testf,vectorizerf,dff = fonction_vectorisation_countvectorizer(r"C:\Users\franc\AutoML\df_train_post_trad.joblib",colonne_a_vectorise,i)
    for j in liste_kbest:
      X_train, X_test, y_train, y_test,vectorizer,df = X_trainf, X_testf, y_trainf, y_testf,vectorizerf,dff
      sel = SelectKBest(k=j)
      sel.fit(np.asarray(X_train),y_train)
      X_train = sel.transform(np.asarray(X_train))
      X_test = sel.transform(np.asarray(X_test))
      debut = time.time()
      rdf.fit(np.asarray(X_train),y_train)
      y_pred = rdf.predict(np.asarray(X_test))
      accuracy = accuracy_score(y_test, y_pred)
      f1_weighted = f1_score(y_test, y_pred, average='weighted')
      f1_macro = f1_score(y_test, y_pred, average='macro')
      fin = time.time()
      duree = fin - debut
      model_scores = {
            "Max_features": f"{i}",
            "K_best":f"{j}",
            "Accuracy": accuracy,
            "F1_weighted": f1_weighted,
            "F1_macro": f1_macro,
            "Duree en sec": duree
        }
      score.append(model_scores)
  df_score = pd.DataFrame(score)
  df = pd.concat([df_import, df_score], ignore_index=True)
  return df

In [27]:
joblib_path = r"C:\Users\franc\AutoML\df_maxfeatures_kbest.joblib"

colonne_a_vectorise = "mots_stem_sans_chiffres"
liste_max_features =[9000]
liste_kbest = [1000,2000,3000,4000,5000,6000,7000]

df_maxfeatures_kbest = recherche_maxfeatures_kbest(joblib_path,colonne_a_vectorise,liste_max_features,liste_kbest)
dump(df_maxfeatures_kbest,r"C:\Users\franc\AutoML\df_maxfeatures_kbest.joblib")

récupération du df existant


['C:\\Users\\franc\\AutoML\\df_maxfeatures_kbest.joblib']

In [2]:
df_maxfeatures_kbest = load(r"C:\Users\franc\AutoML\df_maxfeatures_kbest.joblib")
df_maxfeatures_kbest

,Max_features,K_best,Accuracy,F1_weighted,F1_macro,Duree en sec
0,1000,100,0.568095,0.596247,0.548110,17.682029
1,1000,200,0.632138,0.655485,0.613806,31.827176
2,1000,300,0.671006,0.689518,0.645397,44.603908
3,1000,400,0.687400,0.702872,0.657954,59.463248
4,1000,500,0.702689,0.707845,0.663419,66.754567
...,...,...,...,...,...,...
151,8000,3000,0.768083,0.768647,0.747172,85.092173
152,8000,4000,0.771706,0.771942,0.752257,97.973827
153,8000,5000,0.773978,0.774060,0.754647,111.763977
154,8000,6000,0.777478,0.776706,0.758492,128.356641


In [11]:
import plotly.express as px
fig = px.scatter(df_maxfeatures_kbest, x="F1_weighted", y="F1_macro", color='K_best')
fig.show()

In [12]:
fig = px.scatter(df_maxfeatures_kbest, x="K_best", y="F1_macro", color='Max_features')
fig.show()

In [21]:
fig = px.scatter(df_maxfeatures_kbest, x="K_best", y="F1_weighted", 
                 color='Max_features')
fig.update_layout(
    title={
        'text': "Influence de k et Max_features sur f1_weight",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [7]:
fig = px.scatter(df_maxfeatures_kbest, x="Max_features", y="F1_weighted", color='K_best')
fig.show()

In [14]:
df_maxfeatures_kbest["Max_features"]=df_maxfeatures_kbest["Max_features"].astype("int")

In [16]:
df_maxfeatures_kbest.loc[(df_maxfeatures_kbest["F1_weighted"]>0.77)&(df_maxfeatures_kbest["Max_features"]<=4000)]

,Max_features,K_best,Accuracy,F1_weighted,F1_macro,Duree en sec
82,4000,3000,0.772136,0.772812,0.750719,84.059209
83,4000,3100,0.770294,0.771045,0.749174,85.371924
84,4000,3200,0.771153,0.772074,0.746953,93.747764
85,4000,3300,0.771153,0.771918,0.749508,90.349205
86,4000,3400,0.772136,0.772713,0.750564,97.123296
87,4000,3500,0.771337,0.771887,0.750727,98.807059
88,4000,3600,0.770969,0.771405,0.749628,105.975672
89,4000,3700,0.771215,0.771015,0.749984,101.388523
90,4000,3800,0.770723,0.770742,0.750313,99.500410
91,4000,3900,0.771276,0.771080,0.751835,100.763795
